<a href="https://colab.research.google.com/github/yueqiu2/Evolutionary_Algorithms/blob/main/Opimization%20-%20Gradient%20Descent%20With%20Nesterov%20Momentum%20%2B%20Bayesion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Box-constrained Derivative-Free optimization

In the supporting practice notebook, you were guided through the creation of a trust-region model-based derivative free optimization (DFO) algorithm. In this coursework, you will design your own DFO solver. You have complete freedom in what type of DFO algorithm you want to implement - direct, model-based, evolutionary search, ... Since the takeaway is on optimising chemical engineering systems, we operate under the following assumptions:

- Evaluations are expensive, meaning that the runtime of the algorithms is limited by a fixed evaluation budget rather than a time budget constraint. However, we will not consider algorithms that take longer than 5 minutes for a budget of 1,000 evaluations. Therefore, it might be a good idea to have your algorithm exit, and return the best value found so far if the timing is nearing the 5 minute mark (see for [example](https://stackoverflow.com/questions/889900/accurate-timing-of-functions-in-python))

- We are allowing for budgets ranging between 50 and 1,000 evaluations depending on dimensionalities ranging between 1 and 100

- Input bounds for all dimensions are known


You are given training functions, but you are encouraged to look for your own; This [website](https://www.sfu.ca/~ssurjano/optimization.html) can be useful. Your implementation will be graded based on the best objective evaluation obtained. You will get a minimum of 60% if your implementation beats all benchmarks (Nelder-Mead, Powell, COBYLA), or if you get within $10^{-4}$ of their objective within the allocated budget, and the rest of your grade will be curved by state-of-the-art solvers. A (very bad) optimizer function is given for illustration.

The same marking procedure will then be applied to the testing functions  that you cannot see. This is to make sure that the performance of your optimizer is not 'overfitted' on the training functions. You can prepare for this part by making sure your algorithm still performs well on some of the test functions in the above link. I will make sure that the type of problems remains similar when going from the training to test functions, i.e., I will not test the functions on highly nonconvex / multimodal functions if there are none given as training functions. In the test functions you will get 40% of the overall marks for beating the benchmarks.

**IMPORTANT**:

- Good coding practice: Make sure your function takes the same inputs and outputs as the illustrative optimizer, and make sure that your implementation is flexible enough to handle different dimensionalities. This is **very important**, as otherwise we have no way to test your algorithms, and you will not get marks for those problems (this is easily mitigated by making sure your algorithms run in the training set and follow same inputs and outputs as the sample function).

- Stochasticity: You are highly encouraged to set random seeds if your implementation involves stochasticity. This is to make sure that your implementation works as expected when marking.

- Packages: please restrict to use numpy, scipy, and GPy. You are allowed to use scipy.optimize to train any surrogates if you wish, but you are not allowed to optimize f() directly, neither to use GPyOpt directly on the functions.

- Runtime: For our inexpensive test functions, your algorithm should finish running in **5** minutes, otherwise it will not be graded. This becomes especially relevant if you decide to pursue supplementary workbook B.


Some packages

In [1]:
!pip install GPy

import random
import time
import numpy as np
import scipy.stats as st
import GPy

from scipy.stats import qmc
from functools import partial
from typing import List, Tuple, Dict
from scipy.optimize import minimize, differential_evolution, show_options


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 959.4/959.4 kB 11.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.3/71.3 kB 7.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for GPy: filename=GPy-1.10.0-cp310-cp310-linux_x86_64.whl size=3410102 sha256=b0be02c4e64957894d894d8cde8464d9524bd5777aa4e2ac4e680f89c6aca883
  Stored in directory: /root/.cache/pip/wheels/27/bd/9f/82ab4216eae088cba864ca0dc1d75699bd4bf6823790fb2f77
  Created wheel for paramz: filename=paramz-0.9.5-py3-none-any.whl size=102544 sha256=bb410593b020d44b83ed1150d4d8eda991925bfde647c19912c362e585b7f519
  Stored in directory: /root/.cache/pip/wheels/d2/ef/9c/da9ceef7d0ff5287c24365844fc394852c2b79ac3fcf33bf8b
Successfully built GPy paramz


Some test functions

In [ ]:
def f_d2(x: List[float]) -> float:
  '''
  f is the "black-box" function that needs to be optimized
  Input:
    x: list of size N_x/ array of size (N_x,)
  Output:
    float: function evaluation associated with x
  '''
  x = np.array(x)
  return (1-x[0])**2 + 100*(x[1]-x[0]**2)**2

def f_d3(x: List[float]) -> float:
  '''
  f is the "black-box" function that needs to be optimized
  Input:
    x: list of size N_x/ array of size (N_x,)
  Output:
    float: function evaluation associated with x
  '''
  x = np.array(x)
  return (1-x[0])**2 + 100*(x[1]-x[0]**2)**2 + x[2]**2

A dummy optimizer function. Please keep the function name as 'optimizer'

In [ ]:
# 3 NAG
def NAG(f, bounds, beta, N_x, NAGbudget, initial_x):
    # initialize problem
    # bounds = np.array(bounds)
    x0 = np.array(initial_x)
    # x0 = np.add(bounds[:,1], np.add(bounds[:,0],0.1))/2
    x      = np.copy(x0)

    iter_i = 0
    grad_i = 1e-4*10

    # optimization loop
    v_prev = 0      # initialize at zero to get normal GD-momentum at first step
    while np.sum(np.abs(grad_i)) > 1e-4 and iter_i < NAGbudget:
        grad_i  = grad_f(f,x,N_x)                                 # compute gradient at current position

        x_tilde    = x + beta * v_prev                            # nesterov modified position
        grad_tilde = grad_f(f, x_tilde, N_x)                      # compute gradient of function at x_tilde
        lr         = line_search(grad_tilde, x, f, A=0.1, B=0.8)  # compute learning rate using line search
        v          = nesterov(grad_tilde, v_prev, lr, beta)       # compute momentum

        x       = x + v                                           # compute step
        v_prev  = v                                               # update previous momentum term
        iter_i += 1

    # final optimum
    minimum = f(x)
    print('Iterations: ', iter_i)
    print('Optimal x : ', x)
    print('Minimal f : ', minimum)
    print('Final grad: ', grad_i)

    return minimum

In [ ]:
def grad_f(f, x, N_x):
    # x = [ ], x.shape = (n,), x.shape[0] = n
    # dim = x.shape[0]
    # epsilon: square of smallest float
    # eps  = np.sqrt(np.finfo(float).eps)
    eps = np.sqrt(np.finfo(float).eps)
    # original [[a b]]
    grad = np.zeros((1,N_x))
    # change grad to []
    grad = grad[0]

    for i in range(N_x):

        # e for each dimension with [[]]
        e = np.zeros((1,N_x))
        e[0,i] = eps
        # change e to []
        e = e[0]

        # gradient
        # already change x and e to []
        grad_approx = (f(x+e/2)-f(x-e/2))/eps
        # change from grad[0,i] to grad[i]
        grad[i] = grad_approx

    return grad # grad in []

In [ ]:
def line_search(grad_i, x, f, A, B):
    iter = 0
    lr   = 1

    while f(x - lr*grad_i) > f(x) - A*lr*np.dot(grad_i, grad_i):
        lr    = B*lr
        iter += 1

    return lr  # only return lr

In [ ]:
def nesterov(grad_tilde, v_prev, lr, beta):
    '''
    Momentum function
    INPUTS:
        grad_tilde  : Gradient of function at nesterov modified position
        v_prev      : velocity value at the previous position
        beta        : Momentum hyperparameter
    OUTPUTS:
        v           : Velocity term
    '''
    v = beta * v_prev - lr * grad_tilde
    return v

In [ ]:
'''Confidence bound-based active learning function'''
def CB_learning(x, x_dim, m):
    '''Active learning function based on expected improvement

       This function selects a new sample from candidate pool to enrich the current training dataset.
       The sample gets selected if it has the maximum expected improvement value.'''

    kappa = 2.9

    # 1-return the mean and variance of the prediction
    mean, std = m.predict(np.array([x]))

    # 2-Calculate the CB values
    CB = mean - kappa * std

    return CB

In [ ]:
def CB_opt(x, x_dim, m):
    CB = CB_learning(x, x_dim, m)
    CB = CB.item()
    return CB

In [ ]:
def GP_inference(x,m):
	# Returns the mean and standard deviation of Gaussian process prediction
	mean,std = m.predict(np.array([x]))
	return mean,std

In [ ]:
def GP_obj(x,m):
  '''
	Description:
	Returns the mean prediction of a Gaussian process at a point, x.

	Inputs:
	x - input to be evaluated as either a list or 1-D numpy array
	m - model over which the mean is evaluated

	Outputs:
	mean - the mean of the Gaussian process posterior at x.
	'''
  mean,std = GP_inference(x,m)
  mean = mean.item()
  std = std.item()
  return mean

In [ ]:
def BayesianOpt(f, N_x: int, bounds: List[Tuple[float]], BObudget: int = 100, n_s: int = 100) -> float:

  T0=time.time()

  x_dim = N_x

  '''create initial point x_0'''
  x_0 = []
  for i in range(x_dim):
    a = 0
    x_0.append(a)

  '''create training points for GP'''

  x_train = [] # list for initial training points
  y_train = []
  x_real_list = [] # list for x value of all sample points
  y_real_list = [] # list for y value of all sample points

  bounds = np.array(bounds)
  x_range = bounds[:,1] - bounds[:,0]

  # sampling training points
  sampler = qmc.LatinHypercube(d=x_dim)
  sample = sampler.random(n=n_s)
  l_bounds = bounds[:,0].tolist()
  u_bounds = bounds[:,1].tolist()
  x_train = qmc.scale(sample, l_bounds, u_bounds)
  y_train = np.array([[f(xi)] for xi in x_train])

  '''Main loop to update GP and find the most promising optimal value'''

  n_iter = BObudget - n_s

  for i in range(n_iter):
    # train GPR model
    kernel = GPy.kern.RBF(input_dim=x_dim, ARD=True)
    m = GPy.models.GPRegression(x_train, y_train, kernel )
    m.optimize_restarts(num_restarts = 30, verbose=False)

    # find the next point according confidence bound strategy
    CB_sample = minimize(CB_opt,x_0, args=(x_dim, m), method = 'Powell', bounds=bounds)
    sample = CB_sample.x
    sample_pred, sample_std = GP_inference(sample,m)

    # avoid zero standard deviation sample point
    if sample_std == 0:
       sample = sample + np.random.uniform(-0.5,0.5)*((bounds[0,1]-bounds[0,0])/4)
       # clip x to make it remain in the bound through symmetry
       sample_clip = np.minimum(bounds[:,1], np.maximum(bounds[:,0], sample))
       new_sample = 2 * sample_clip - sample
    else:
       new_sample = sample

    # Calculate the true y value of the new sample
    y_sample = f(new_sample)

    print('new_sample = ', new_sample)
    print('y_sample = ', y_sample)

    # Enrich training dataset
    x_train = np.vstack((x_train,new_sample))
    y_train = np.vstack((y_train,y_sample))

    # time limit
    if time.time()-T0 >298:
            break

  best_index = np.argmin(y_train)
  y_best = y_train[best_index]
  x_best = x_train[best_index,:]
  print('x_best', x_best)
  print('y_best', y_best)

  return x_best, y_best

In [ ]:
def optimizer(f, N_x: int, bounds: List[Tuple[float]], N: int = 100) -> float:
  '''
  Optimizer aims to optimize a black-box function 'f' using the dimensionality
  'N_x', and box-'bounds' on the decision vector
  Input:
    f: function: taking as input a list of size N_x and outputing a float
    N_x: int: number of dimensions
    N: int: optional: Evaluation budget
    bounds: List of size N where each element i is a tuple conisting of 2 floats
            (lower, upper) serving as box-bounds on the ith element of x
  Return:
    lowest value found for f, f_min
  '''

  if N_x != len(bounds):
    raise ValueError('Nbr of variables N_x does not match length of bounds')

  ### Your code here

  if N <= 50:
     BObudget = 30
     n_s = 20
     x_best, y_best = BayesianOpt(f, N_x, bounds, BObudget, n_s)
  else:
     BObudget = 0
     bounds = np.array(bounds)
     x_best = np.add(bounds[:,1], np.add(bounds[:,0],0.1))/2

  beta = 0.62
  NAGbudget = N - BObudget
  initial_x = x_best
  y_best = NAG(f, bounds, beta, N_x, NAGbudget, initial_x)


  return y_best


Although you are not allowed to use packaged solvers to directly optimize the functions in your implementation, you are still encouraged to compare your results against state-of-the-art DFO implementations, such as py-bobyqa, GPyOpt, Entmoot, or various methods within scipy.optimize.

Note that minimize() requires slightly different inputs to our optimizer functions, such as an initial guess

In [ ]:
N_x = 2
budget = 50
bounds_RB = [(-2., 2.)]*N_x
x0 = [0, 0]

result_benchmark1 = minimize(
    f_d2, x0,
    bounds=bounds_RB,
    method='Nelder-Mead',
    options={'maxfev': budget},
    )

result_benchmark2 = minimize(
    f_d2, x0,
    bounds=bounds_RB,
    method='Powell',
    options={'maxfev': budget},
    )

result_benchmark3 = minimize(
    f_d2, x0,
    bounds=bounds_RB,
    method='COBYLA',
    options={'maxiter': budget},
    )

t0 = time.time()
result_dummy = optimizer(f_d2, N_x, bounds_RB, budget)
t = time.time()
assert (t - t0 < 150), "Ensure your optimizer finishes in 5 minutes"

score = int(result_dummy < result_benchmark1.fun) + int(result_dummy < result_benchmark2.fun) + int(result_dummy < result_benchmark3.fun)

print(f"Nelder-Mead reached an optimum of {result_benchmark1.fun:.3f}")
print(f"Powell reached an optimum of {result_benchmark2.fun:.3f}")
print(f"COBYLA reached an optimum of {result_benchmark3.fun:.3f}")

print(f"Your optimizer reached an optimum of {result_dummy:.3f}, beating {score}/3 benchmarks")


 /usr/local/lib/python3.8/dist-packages/scipy/optimize/_minimize.py:544: RuntimeWarning:Method COBYLA cannot handle bounds.
 /usr/local/lib/python3.8/dist-packages/GPy/kern/src/stationary.py:243: RuntimeWarning:invalid value encountered in true_divide
 /usr/local/lib/python3.8/dist-packages/GPy/kern/src/stationary.py:137: RuntimeWarning:overflow encountered in square
 /usr/local/lib/python3.8/dist-packages/GPy/kern/src/stationary.py:138: RuntimeWarning:invalid value encountered in add
 /usr/local/lib/python3.8/dist-packages/GPy/kern/src/rbf.py:76: RuntimeWarning:invalid value encountered in multiply
 /usr/local/lib/python3.8/dist-packages/GPy/kern/src/stationary.py:166: RuntimeWarning:overflow encountered in true_divide


new_sample =  [-2.00000000e+00 -4.54217433e-04]
y_sample =  1609.363390075178
new_sample =  [-1.93033755 -1.99973557]
y_sample =  3287.2242129832825
new_sample =  [-1.18364583 -0.53763354]
y_sample =  380.60507124439954
new_sample =  [0.15946914 2.        ]
y_sample =  390.5990004412579
new_sample =  [-0.38610954  0.97190309]
y_sample =  69.62498887620583
new_sample =  [0.83303858 2.        ]
y_sample =  170.6036804379903
new_sample =  [-1.23389296 -0.815121  ]
y_sample =  551.4336547917379
new_sample =  [-1.37053116 -1.99973557]
y_sample =  1509.5785803033261
new_sample =  [0.64123211 0.4149118 ]
y_sample =  0.13010805444209383
new_sample =  [-0.93252103 -0.61268713]
y_sample =  223.45080863569677
x_best [0.64123211 0.4149118 ]
y_best [0.13010805]
Iterations:  20
Optimal x :  [0.73271561 0.53378178]
Minimal f :  0.07239599170346978
Final grad:  [-0.15650273 -0.26263739]
Nelder-Mead reached an optimum of 0.438
Powell reached an optimum of 0.527
COBYLA reached an optimum of 0.408
Your o

# Training sets (30 marks)
Two-dimensional datasets


In [ ]:
def Matyas(x: List[float]) -> float:
  x = np.array(x)
  return .26*(x[0]**2 + x[1]**2) - .48*x[0]*x[1]
bounds_Matyas = [(-10., 7.), (-7., 10.)]

def Bohachevsky(x: List[float]) -> float:
  x = np.array(x)
  out = x[0]**2+2*x[1]**2-.3*np.cos(3*np.pi*x[0])-.4*np.cos(4*np.pi*x[1]) + .7
  return out
bounds_Bohachevsky = [(-7., 10.), (-10., 7.)]

train_2d_list = [f_d2, Matyas, Bohachevsky]
bounds_train_2d = [bounds_RB, bounds_Matyas, bounds_Bohachevsky]
budget_list = [50, 50, 50]
dim_list = [2, 2, 2]

In [ ]:
all_results = {'benchmark1': [], 'benchmark2': [], 'benchmark3': [], 'dummy': [],}

for iteration in zip(bounds_train_2d, train_2d_list, budget_list, dim_list):

  bounds, function, budget, dim = iteration

  benchmark1 = minimize(
    function, x0,
    bounds=bounds,
    method='Nelder-Mead',
    options={'maxfev': budget},
    )

  benchmark2 = minimize(
    function, x0,
    bounds=bounds,
    method='Powell',
    options={'maxfev': budget},
    )

  benchmark3 = minimize(
    function, x0,
    bounds=bounds,
    method='COBYLA',
    options={'maxiter': budget},
    )

  t0 = time.time()
  dummy = optimizer(function, N_x, bounds, budget)
  t = time.time()
  assert (t - t0 < 300), "Ensure your optimizer finishes in 5 minutes"

  all_results['benchmark1'] += [benchmark1.fun]
  all_results['benchmark2'] += [benchmark2.fun]
  all_results['benchmark3'] += [benchmark3.fun]
  all_results['dummy'] += [dummy]


print(f"Optima reached {all_results}")

new_sample =  [0.58249997 1.99993758]
y_sample =  275.943958650518
new_sample =  [-0.68905379  0.42976374]
y_sample =  3.0556852820719715
new_sample =  [0.27743093 0.07464792]
y_sample =  0.5226442996797707
new_sample =  [-1.99996566 -2.        ]
y_sample =  3608.8349671008446
new_sample =  [-1.99996566 -2.        ]
y_sample =  3608.8349671008446
new_sample =  [1.99993758 1.99993758]
y_sample =  400.9249726874768


 /usr/local/lib/python3.8/dist-packages/GPy/kern/src/stationary.py:138: RuntimeWarning:overflow encountered in multiply
 /usr/local/lib/python3.8/dist-packages/GPy/kern/src/stationary.py:138: RuntimeWarning:overflow encountered in add


new_sample =  [1.999899   1.99993758]
y_sample =  400.86317901302715
new_sample =  [-2.         -0.19962562]
y_sample =  1772.6855268091817
new_sample =  [1.99999996 1.2438537 ]
y_sample =  760.6341475971664
new_sample =  [1.58007453 0.49024849]
y_sample =  402.89537421759525
x_best [0.27743093 0.07464792]
y_best [0.5226443]
Iterations:  20
Optimal x :  [0.55530948 0.30797092]
Minimal f :  0.1977654743313061
Final grad:  [-1.88012671  0.89074002]
new_sample =  [0.01750009 0.01650355]
y_sample =  1.1810752480820937e-05
new_sample =  [-0.00744805 -0.01297696]
y_sample =  1.1814024082722157e-05
new_sample =  [0.00073974 0.00151859]
y_sample =  2.026517571264689e-07
new_sample =  [-0.00011083  0.00173595]
y_sample =  8.790544514172362e-07
new_sample =  [-2.83362713e-05  1.90281483e-04]
y_sample =  1.221069314292257e-08
new_sample =  [ 0.00333035 -0.00499213]
y_sample =  1.734351012656867e-05
new_sample =  [-3.80459966e-06  4.74199081e-03]
y_sample =  5.855147601783365e-06
new_sample =  [0.

Increasing dimensionality unimodal

In [ ]:
dims = [2, 6, 10, 25]

def Zakharov(d, x):
    return np.sum([x[i]**2 for i in range(d)]) + np.sum([(0.5*i*x[i])**2 for i in range(d)]) + np.sum([(0.5*i*x[i])**4 for i in range(d)])

Zakharov_list = [partial(Zakharov, dim) for dim in dims]
bounds_Zakharov = [[(-5, 10)]*dim for dim in dims]
print(bounds_Zakharov)
budget_Zakharov = [50, 100, 100, 500, 1000, 1000]

[[(-5, 10), (-5, 10)], [(-5, 10), (-5, 10), (-5, 10), (-5, 10), (-5, 10), (-5, 10)], [(-5, 10), (-5, 10), (-5, 10), (-5, 10), (-5, 10), (-5, 10), (-5, 10), (-5, 10), (-5, 10), (-5, 10)], [(-5, 10), (-5, 10), (-5, 10), (-5, 10), (-5, 10), (-5, 10), (-5, 10), (-5, 10), (-5, 10), (-5, 10), (-5, 10), (-5, 10), (-5, 10), (-5, 10), (-5, 10), (-5, 10), (-5, 10), (-5, 10), (-5, 10), (-5, 10), (-5, 10), (-5, 10), (-5, 10), (-5, 10), (-5, 10)]]


In [ ]:
all_results = {'benchmark1': [], 'benchmark2': [], 'benchmark3': [], 'dummy': [],}

for iteration in zip(bounds_Zakharov, Zakharov_list, budget_Zakharov, dims):

  bounds, function, budget, dim = iteration
  x0 = [np.mean(list(b)) for b in bounds]

  benchmark1 = minimize(
    function, x0,
    bounds=bounds,
    method='Nelder-Mead',
    options={'maxfev': budget},
    )

  benchmark2 = minimize(
    function, x0,
    bounds=bounds,
    method='Powell',
    options={'maxfev': budget},
    )

  benchmark3 = minimize(
    function, x0,
    bounds=bounds,
    method='COBYLA',
    options={'maxiter': budget},
    )

  t0 = time.time()
  dummy = optimizer(function, dim, bounds, budget)
  t = time.time()
  # assert (t - t0 < 300), "Ensure your optimizer finishes in 5 minutes"
  print('time =', t-t0)

  all_results['benchmark1'] += [benchmark1.fun]
  all_results['benchmark2'] += [benchmark2.fun]
  all_results['benchmark3'] += [benchmark3.fun]
  all_results['dummy'] += [dummy]

print(f"Optima reached {all_results}")

new_sample =  [-4.99968275  2.03302892]
y_sample =  31.231048163087394
new_sample =  [9.99927514 5.02144527]
y_sample =  171.24113274063245
new_sample =  [ 9.99999996 -4.99999999]
y_sample =  170.3124989311118
new_sample =  [-0.6858968   0.95614581]
y_sample =  1.6654597332867653
new_sample =  [0.13852493 0.00588606]
y_sample =  0.019232464685271875
new_sample =  [ 9.99999998 -2.85185981]
y_sample =  114.3005917264356
new_sample =  [-1.92658629  6.90090063]
y_sample =  204.98325855870448
new_sample =  [ 1.79079069e-03 -1.76103707e-06]
y_sample =  3.2069351564625285e-06
new_sample =  [ 0.00042005 -0.00107325]
y_sample =  1.6162776597598487e-06
new_sample =  [ 0.00014971 -0.00205924]
y_sample =  5.322977806074881e-06
x_best [ 0.00042005 -0.00107325]
y_best [1.61627766e-06]
Iterations:  8
Optimal x :  [-3.72747379e-06 -2.80479316e-04]
Minimal f :  9.834970282651013e-08
Final grad:  [-2.01123997e-05 -3.92432254e-05]
time = 48.409862756729126
Iterations:  99
Optimal x :  [ 4.42169138e-12  4

Increasing dimensionality Rosenbrock

In [ ]:
def RB_decoupled(dim, x):
  return np.sum([100*(x[i]**2 - x[i-1])**2+(x[i-1]-1)**2 for i in range(1,dim)])

RB_list = [partial(RB_decoupled, dim) for dim in dims]
bounds_RB = [[(-2, 2)]*dim for dim in dims]
budget_RB = [50, 100, 100, 500, 1000, 1000]

In [ ]:
all_results = {'benchmark1': [], 'benchmark2': [], 'benchmark3': [], 'dummy': [],}

for iteration in zip(bounds_RB, RB_list, budget_RB, dims):

  bounds, function, budget, dim = iteration
  x0 = [np.mean(list(b)) for b in bounds]

  benchmark1 = minimize(
    function, x0,
    bounds=bounds,
    method='Nelder-Mead',
    options={'maxfev': budget},
    )

  benchmark2 = minimize(
    function, x0,
    bounds=bounds,
    method='Powell',
    options={'maxfev': budget},
    )

  benchmark3 = minimize(
    function, x0,
    bounds=bounds,
    method='COBYLA',
    options={'maxiter': budget},
    )

  t0 = time.time()
  dummy = optimizer(function, dim, bounds, budget)
  t = time.time()
  assert (t - t0 < 300), "Ensure your optimizer finishes in 5 minutes"

  all_results['benchmark1'] += [benchmark1.fun]
  all_results['benchmark2'] += [benchmark2.fun]
  all_results['benchmark3'] += [benchmark3.fun]
  all_results['dummy'] += [dummy]

print(f"Optima reached {all_results}")

new_sample =  [ 2.         -0.49249612]
y_sample =  309.8621976967834
new_sample =  [-1.99996566 -2.        ]
y_sample =  3608.958580220184
new_sample =  [-2.          1.99996566]
y_sample =  3608.835173126837
new_sample =  [-1.20611917  1.99989444]
y_sample =  2714.795049279072
new_sample =  [1.99973557 1.79900672]
y_sample =  153.9395929479596
new_sample =  [-0.01798285  1.44836995]
y_sample =  448.67963444421974
new_sample =  [ 1.99973557 -1.05023452]
y_sample =  81.41427627464248
new_sample =  [ 0.21199461 -0.74475507]
y_sample =  12.362917353063562
new_sample =  [-2.          1.04361058]
y_sample =  963.2681195734003
new_sample =  [-2.         -0.00859377]
y_sample =  409.02954111407536
x_best [0.25767181 0.4789633 ]
y_best [0.63094766]
Iterations:  20
Optimal x :  [0.41029892 0.64010926]
Minimal f :  0.34777861777341623
Final grad:  [-1.74113071  0.6758331 ]
Iterations:  100
Optimal x :  [0.54178877 0.73646068 0.85823887 0.92643748 0.96183835 0.98064409]
Minimal f :  0.3065835246

Test sets (30 marks)

In [ ]:
#@title 2-dimensional test functions (6 marks)

def MCCormick(x: List[float]) -> float:
  x = np.array(x)
  return np.sin(x[0]+x[1]) + (x[0]-x[1])**2 - 1.5*x[0] + 2.5*x[1] + 1
bounds_MCCormick = [(-1.5, 4.), (-3., 4.)]

def THC(x: List[float]) -> float:
  x = np.array(x)
  out = 2*x[0]**2 - 1.05*x[1]**4 + x[0]**6/6 + x[0]*x[1] + x[1]**2
  return out
bounds_THC = [(-3., 2.), (-2., 3.)]

def Branin(x: List[float]) -> float:
  x = np.array(x)
  a = 1 ; b = 5.1/(4*np.pi**2) ; c = 5/np.pi
  r = 6 ; s = 10 ; t =1/(8*np.pi)
  return a*(x[1] - b*x[0]**2 + c*x[0] - r)**2 + s*(1-t)*np.cos(x[0]) + s
bounds_Branin = [(-5., 10.), (0., 15.)]

test_2d_list = [MCCormick, THC, Branin]
bounds_test_2d = [bounds_MCCormick, bounds_THC, bounds_Branin]
budget_list = [50, 50, 50]
dim_list = [2, 2, 2]

In [ ]:
all_results = {'benchmark1': [], 'benchmark2': [], 'benchmark3': [], 'dummy': [],}

for iteration in zip(bounds_test_2d, test_2d_list, budget_list, dim_list):

  bounds, function, budget, dim = iteration
  x0 = [np.mean(list(b)) for b in bounds]

  benchmark1 = minimize(
    function, x0,
    bounds=bounds,
    method='Nelder-Mead',
    options={'maxfev': budget},
    )

  benchmark2 = minimize(
    function, x0,
    bounds=bounds,
    method='Powell',
    options={'maxfev': budget},
    )

  benchmark3 = minimize(
    function, x0,
    bounds=bounds,
    method='COBYLA',
    options={'maxiter': budget},
    )

  t0 = time.time()
  dummy = optimizer(function, dim, bounds, budget)
  t = time.time()
  assert (t - t0 < 300), "Ensure your optimizer finishes in 5 minutes"

  all_results['benchmark1'] += [benchmark1.fun]
  all_results['benchmark2'] += [benchmark2.fun]
  all_results['benchmark3'] += [benchmark3.fun]
  all_results['dummy'] += [dummy]

print(f"Optima reached {all_results}")

new_sample =  [-0.86408555 -1.80515957]
y_sample =  -1.7861283085665098
new_sample =  [-0.80264235 -1.82243502]
y_sample =  -1.8059999575271917
new_sample =  [-0.78612894 -1.80945474]
y_sample =  -1.816528375059442
new_sample =  [-0.62645766 -1.4563971 ]
y_sample =  -1.8842445644355097
new_sample =  [-0.55646817 -1.54876301]
y_sample =  -1.9131128474867878
new_sample =  [-0.5497539  -1.54778919]
y_sample =  -1.9132148064329768
new_sample =  [-0.54935698 -1.54768207]
y_sample =  -1.9132171242505023
new_sample =  [-0.54921702 -1.54761455]
y_sample =  -1.9132178177296209
new_sample =  [-0.54916417 -1.54758663]
y_sample =  -1.91321806451358
new_sample =  [-0.5491128 -1.547596 ]
y_sample =  -1.9132183373194356
x_best [-0.5491128 -1.547596 ]
y_best [-1.91321834]
Iterations:  16
Optimal x :  [-0.54729087 -1.54710583]
Minimal f :  -1.9132229207394715
Final grad:  [-3.91006470e-05  3.65376472e-05]
new_sample =  [0.33379761 0.31212496]
y_sample =  0.4147151805625138
new_sample =  [ 0.99298219 -0

In [ ]:
#@title Increasing dimensionality: Well-behaved (12 marks)

def Styblinski_Tang(dim, x):
    d = dim
    f = 1/2 * np.sum([(x[i]**4 - 16*x[i]**2 + 5*x[i]) for i in range(d)])
    return f

ST_list = [partial(Styblinski_Tang, dim) for dim in dims]
bounds_ST = [[(-5., 5.)]*dim for dim in dims]
budget_ST = [50, 100, 100, 500, 1000, 1000]


In [ ]:
all_results = {'benchmark1': [], 'benchmark2': [], 'benchmark3': [], 'dummy': [],}

for iteration in zip(bounds_ST, ST_list, budget_ST, dims):

  bounds, function, budget, dim = iteration
  x0 = [np.mean(list(b)) for b in bounds]

  benchmark1 = minimize(
    function, x0,
    bounds=bounds,
    method='Nelder-Mead',
    options={'maxfev': budget},
    )

  benchmark2 = minimize(
    function, x0,
    bounds=bounds,
    method='Powell',
    options={'maxfev': budget},
    )

  benchmark3 = minimize(
    function, x0,
    bounds=bounds,
    method='COBYLA',
    options={'maxiter': budget},
    )

  t0 = time.time()
  dummy = optimizer(function, dim, bounds, budget)
  t = time.time()
  assert (t - t0 < 300), "Ensure your optimizer finishes in 5 minutes"

  all_results['benchmark1'] += [benchmark1.fun]
  all_results['benchmark2'] += [benchmark2.fun]
  all_results['benchmark3'] += [benchmark3.fun]
  all_results['dummy'] += [dummy]

print(f"Optima reached {all_results}")

new_sample =  [ 0.033747   -2.61945649]
y_sample =  -37.825352380096554
new_sample =  [ 4.99994039 -1.0237944 ]
y_sample =  114.59430641724492
new_sample =  [0.85351387 4.99969584]
y_sample =  121.51878146964899
new_sample =  [-4.99994039  2.2863576 ]
y_sample =  77.54947903069117
new_sample =  [-2.20125472  2.27811455]
y_sample =  -54.88389022579062
new_sample =  [ 4.99999999 -3.44547346]
y_sample =  91.87974822211706
new_sample =  [ 3.23751009 -2.88524068]
y_sample =  -59.98778883820047
new_sample =  [-3.38352987  2.95932989]
y_sample =  -58.8281976830356
new_sample =  [ 2.28323117 -4.99905441]
y_sample =  77.43303262632219
new_sample =  [ 1.36040166 -0.70979321]
y_sample =  -15.370029421173836
x_best [-2.40903604 -2.55737292]
y_best [-72.93804476]
Iterations:  20
Optimal x :  [-2.90105289 -2.90134441]
Minimal f :  -78.33214220678047
Final grad:  [-0.279459   -0.24749947]
Iterations:  95
Optimal x :  [-2.90353504 -2.90353504 -2.90353502 -2.90353508 -2.9035351  -2.90353498]
Minimal f 

In [ ]:
#@title Increasing dimensionality: nonconvex (12 marks)

def RB_aug(dim, x):
  obj = np.sum([100*(x[i]**2 - x[i-1])**2+(x[i-1]-1)**2 for i in range(1,dim)])

  penalty1 = np.sum([max(0,  (x[i-1] - 1)**3 - x[i] + 1)**2 for i in range(1,dim)])
  penalty2 = np.sum([max(0,  x[0] + x[1] - 1.8)**2 for i in range(1,dim)])
  return obj + 1000*(penalty1 + penalty2)

RBaug_list = [partial(RB_decoupled, dim) for dim in dims]
bounds_RBaug = [[(-2, 2)]*dim for dim in dims]
budget_RBaug = [50, 100, 100, 500, 1000, 1000]

In [ ]:
all_results = {'benchmark1': [], 'benchmark2': [], 'benchmark3': [], 'dummy': [],}

for iteration in zip(bounds_RBaug, RBaug_list, budget_RBaug, dims):

  bounds, function, budget, dim = iteration
  x0 = [np.mean(list(b)) for b in bounds]

  benchmark1 = minimize(
    function, x0,
    bounds=bounds,
    method='Nelder-Mead',
    options={'maxfev': budget},
    )

  benchmark2 = minimize(
    function, x0,
    bounds=bounds,
    method='Powell',
    options={'maxfev': budget},
    )

  benchmark3 = minimize(
    function, x0,
    bounds=bounds,
    method='COBYLA',
    options={'maxiter': budget},
    )

  t0 = time.time()
  dummy = optimizer(function, dim, bounds, budget)
  t = time.time()
  assert (t - t0 < 300), "Ensure your optimizer finishes in 5 minutes"

  all_results['benchmark1'] += [benchmark1.fun]
  all_results['benchmark2'] += [benchmark2.fun]
  all_results['benchmark3'] += [benchmark3.fun]
  all_results['dummy'] += [dummy]

print(f"Optima reached {all_results}")

new_sample =  [-2.          0.25738062]
y_sample =  435.9367508336016
new_sample =  [-1.99993746  0.66799822]
y_sample =  607.3690533033723
new_sample =  [ 2.         -1.74732684]
y_sample =  111.91272415921469
new_sample =  [-0.88802607 -0.20310843]
y_sample =  89.92060762087692
new_sample =  [1.99973557 1.9997013 ]
y_sample =  400.6274370997357
new_sample =  [ 0.05558274 -0.66718208]
y_sample =  16.06678122987208
new_sample =  [ 0.47091144 -0.66849321]
y_sample =  0.33767047942785927
new_sample =  [ 0.213014   -0.45657836]
y_sample =  0.6214173941263297
new_sample =  [2.        1.2528534]
y_sample =  19.520830508136115
new_sample =  [1.26498928 1.12427703]
y_sample =  0.0703174187698884
x_best [1.26498928 1.12427703]
y_best [0.07031742]
Iterations:  20
Optimal x :  [1.15776059 1.07632665]
Minimal f :  0.024940022986035208
Final grad:  [-0.21987565  1.16123184]
Iterations:  100
Optimal x :  [0.54178877 0.73646068 0.85823887 0.92643748 0.96183835 0.98064409]
Minimal f :  0.306583524646

# Test sets (30 marks)

# Supplementary problem (40 marks)

Choose *ONLY ONE* of the two supplementary problems (A *OR* B) for the remaining marks.
You will build on your solution to the above problem (or build another solution from scratch) by considering one of the following extensions to your algorithm:

- Discrete variables
- High-dimensional problems with low effective dimensionality



##A) Discrete variables (40 marks)

Like in conventional optimization, the presence of discrete decision variables complicates DFO significantly because they prevent the use of local information that many DFO methods rely on. If you choose this supplementary problem, you will build on your previous algorithm (or construct a new one) that takes as additional input a dictionary of the integer variable keys and a list of possible values they can take on, and returns the best possible solution found in the given evaluation budget.

You may find the below dummy optimizer and test function useful for illustration:

In [ ]:
#@title Discrete test function (5 marks)
def f_discrete(x: List[float], y: Dict[str, int]):
  np.random.seed(0)
  keys = list(y.keys())
  continuous = np.sum([x_**2 for x_ in x])
  discrete = np.sum([np.random.normal()*y[key] for key in keys])
  bilinear = np.random.choice(x)*y[random.choice(keys)]
  return continuous + discrete + bilinear

bounds = [(-2, 4), (-4, 2)] ; N_x = 2
integers_dict = {'y1': [-1, 0, 1], 'y2': [-1, 1],}

In [ ]:
def optimizer_discrete(f, N_x: int,
                       bounds: List[Tuple[float]],
                       integers: Dict[str, List[int]],
                       N: int = 100) -> float:
  '''
  Optimizer aims to optimize a black-box function 'f' using the dimensionality
  'N_x', and box-'bounds' on the decision vector
  Input:
    f: function: taking as input a list of size N_x and outputing a float
    N_x: int: number of CONTINUOUS dimensions
    N: int: optional: Evaluation budget
    bounds: List of size N where each element i is a tuple conisting of 2 floats
            (lower, upper) serving as box-bounds on the ith element of x
    integers:
  Return:
    lowest value found for f, f_min
  '''
  if N_x != len(bounds):
    raise ValueError('Nbr of variables N_x does not match length of bounds')

  ### Your code here
  x = [np.mean(bounds[i]) for i in range(N_x)]
  keys = integers.keys()
  y = {key: np.random.choice(integers[key]) for key in keys}
  f_best = f(x, y)
  ###

  return f_best

t0 = time.time()
test_result = optimizer_discrete(
    f_discrete, len(bounds), bounds, integers_dict, N=100,
)
t = time.time()

In [ ]:
#@title Other form of discrete test function
keys = list(integers_dict.keys())
bounds_discr = [(-2, 4), (-4, 2), (-1, 1), (-1, 1)]
x0_discr = np.array([np.mean(list(b_)) for b_ in bounds_discr])

def f_discr_cont(x_extended):
  ## this function is the same as f_discrete() but where the discrete variables
  ## are relaxed to be continuous so you can check against
  ## continuous scipy.optimize.minimize() DFO solvers
  np.random.seed(0)
  continuous = np.sum([x_**2 for x_ in x_extended[:2]])
  discrete = np.sum([np.random.normal()*x_ for x_ in x_extended[2:]])
  x_ = np.random.choice(x_extended[:2])
  idx = keys.index(random.choice(keys))
  bilinear = x_*x_extended[2+idx]
  return continuous + discrete + bilinear

In [ ]:
budget = 10000
benchmark = minimize(
    f_discr_cont, x0_discr,
    bounds=bounds_discr,
    method='Powell',
    options={'maxfev': budget},
    )

x_opt = benchmark.x.copy()
x_opt[2:] = np.round(x_opt[2:])
actual_f = f_discrete(x_opt[:2], {'y1': x_opt[-2], 'y2': x_opt[-1]})
print(f"After rounding of the discrete variables, Powell reaches an optimum of {actual_f:.3f} at {x_opt}")

t0 = time.time()
result_dummy = optimizer_discrete(f_discrete, N_x, bounds, integers_dict, budget)
t = time.time()
assert (t - t0 < 300), "Ensure your optimizer finishes in 5 minutes"
print(f"Your optimizer reaches an optimum of {result_dummy:.3f}")




After rounding of the discrete variables, Powell reaches an optimum of -2.396 at [ 0.09622247  0.40375334 -1.         -1.        ]
Your optimizer reaches an optimum of 2.400


Using discrete derivative-free optimization solvers for hyperparameter tuning

Starting with the wine regression playground for MLPClassifier hyperparameter tuning

In [ ]:
import itertools
import pandas as pd

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor, MLPClassifier
from sklearn.model_selection import (GridSearchCV, StratifiedShuffleSplit, cross_validate)

In [ ]:
#@title Wine classification example (10 marks)
 # ============= Wine classification =============
df = pd.read_csv("https://raw.githubusercontent.com/dsrscientist/DSData/master/winequality-red.csv")
X = df.values[:,:-1]
y = df.values[:,-1]
cv_N = 5

def wine_classification(continous, discrete):
  pipe_all = make_pipeline(
      StandardScaler(),
      MLPClassifier(
          hidden_layer_sizes=(discrete['hidden1'], discrete['hidden2']),
          learning_rate_init=continuous[0],
          random_state=0,
          early_stopping=bool(discrete['early_stopping']),
          ),
      )
  cv = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=42)
  scores = cross_validate(pipe_all, X, y, cv=cv, return_train_score=True)
  return np.mean(scores['test_score'])

Grid Search

In [ ]:
pipe = make_pipeline(
      StandardScaler(),
      MLPClassifier(
          hidden_layer_sizes=(100, 50),
          learning_rate_init=1,
          random_state=0,
          early_stopping=True,
          ),
      )

hidden_layers1 = [80, 100, 120]
hidden_layers2 = [40, 50, 60]
learning_rates = np.logspace(-1, 1, 6)
param_grid = {
    'mlpclassifier__hidden_layer_sizes': list(itertools.product(hidden_layers1, hidden_layers2)),
    'mlpclassifier__learning_rate_init': learning_rates,
}

cv = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=42)
grid = GridSearchCV(pipe, param_grid=param_grid, cv=cv)
grid.fit(X, y)

print(
    "The best parameters are %s with a score of %0.2f"
    % (grid.best_params_, grid.best_score_)
)

The best parameters are {'mlpclassifier__hidden_layer_sizes': (120, 40), 'mlpclassifier__learning_rate_init': 0.1} with a score of 0.62


In [ ]:
continuous = [grid.best_params_['mlpclassifier__learning_rate_init']]
discrete = {
    'hidden1': grid.best_params_['mlpclassifier__hidden_layer_sizes'][0],
    'hidden2': grid.best_params_['mlpclassifier__hidden_layer_sizes'][1],
    'early_stopping': 1,
    }

gridsearch_score = wine_classification(continuous, discrete)
print(f"Gridsearch reached a score of {gridsearch_score:.3f}")

bounds = [(0.1, 10)] ; N_x = 1 ; budget = 100
integers_dict = {
    'hidden1': hidden_layers1,
    'hidden2': hidden_layers2,
    'early_stopping': [0,1],
    }

t0 = time.time()
dummy = optimizer_discrete(wine_classification, N_x, bounds, integers_dict, budget)
t = time.time()
assert (t - t0 < 300), "Ensure your optimizer finishes in 5 minutes"

print(f"Your optimizer reached an optimum of {dummy:.3f}")

Gridsearch reached a score of 0.621
Your optimizer reached an optimum of 0.582


Revisiting ML models that make sense. Doing extensive GridSearchCV is too expensive on this problem

In [ ]:
#@title Load VLE data
directory_2 = "https://raw.githubusercontent.com/OptiMaL-PSE-Lab/ML-course/master/VLE%20prediction/Data.xlsx"
directory_1 = "https://raw.githubusercontent.com/OptiMaL-PSE-Lab/ML-course/master/VLE%20prediction/Test-Dataset%20(no%20noise).xlsx"
df_test = pd.read_excel(directory_1, sheet_name='Phase Envelope')
df = pd.read_excel(directory_2, sheet_name='Phase Envelope')
df_array = df.values
df_test_array = df_test.values
X_idx = [0]
yP_idx = 1
yliq_idx = 2
yvap_idx = 3
X_train = df_array[:, X_idx]
X_test = df_test_array[:, X_idx]
y_liq_train = df_array[:, yliq_idx].squeeze()
y_liq_test = df_test_array[:, yliq_idx].squeeze()
y_vap_train = df_array[:, yvap_idx].squeeze()
y_vap_test = df_test_array[:, yvap_idx].squeeze()
yP_train = df_array[:, yP_idx].squeeze()
yP_test = df_test_array[:, yP_idx].squeeze()

df_critical = pd.read_excel(directory_1, sheet_name='Critical Point')
T_c = df_critical['Temperature / K'][0]
P_c = df_critical['Pressure / Pa'][0]
rho_c = df_critical['Density / (mol m^-3)'][0]


In [ ]:
#@title DFO for hyperparameter tuning (15 marks)
def physical_ML(continuous, discrete, print_corr=True, print_pred=True):

  pipe_liq = make_pipeline(
      StandardScaler(),
      MLPRegressor(
          hidden_layer_sizes=(discrete['liq_hidden1'],discrete['liq_hidden2']),
          learning_rate_init=continuous[0],
          random_state=0,
          )
      )

  pipe_vap = make_pipeline(
      StandardScaler(),
      MLPRegressor(
          hidden_layer_sizes=(discrete['vap_hidden'],),
          learning_rate_init=continuous[1],
          random_state=0,
          )
      )

  pipe_P = make_pipeline(
      StandardScaler(),
      MLPRegressor(
          hidden_layer_sizes=(discrete['P_hidden1'],discrete['P_hidden2']),
          learning_rate_init=continuous[2],
          random_state=0,
          )
      )

  pipe_liq.fit(X_train, y_liq_train)
  pipe_vap.fit(X_train, y_vap_train)
  pipe_P.fit(X_train, yP_train)

  R2_train_liq = pipe_liq.score(X_train, y_liq_train)
  R2_train_vap = pipe_vap.score(X_train, y_vap_train)
  R2_test_liq = pipe_liq.score(X_test, y_liq_test)
  R2_test_vap = pipe_vap.score(X_test, y_vap_test)
  R2_train_T = pipe_P.score(X_train, yP_train)
  R2_test_T = pipe_P.score(X_test, yP_test)

  if print_corr:
    print(f"R squared value on liq test set: {R2_test_liq:.3f}")
    print(f"R squared value on vap test set: {R2_test_vap:.3f}")
    print(f"R squared value on T test set: {R2_test_T:.3f}")

  XP = np.array([ [P] for P in range(int(T_c/2), int(T_c*2))])

  rho_liq_pred = pipe_liq.predict(XP)
  rho_vap_pred = pipe_vap.predict(XP)
  P_pred = pipe_P.predict(XP)
  rho_deviation = np.abs(rho_liq_pred - rho_vap_pred)
  idx = np.where(rho_deviation == np.min(rho_deviation))

  if print_pred:
    print(f"Predicted critical point: T - {float(XP[idx])} ; P - {float(P_pred[idx]):.0f} ; rho_liq - {float(rho_liq_pred[idx]):.0f} ; rho_vap - {float(rho_vap_pred[idx]):.0f}")
    print(f"Real critical point: T {T_c} - P - {P_c:.0f} ; rho_liq - {rho_c:.0f} ; rho_vap - {rho_c:.0f}")
    print("")
  min_corr = -R2_test_liq-R2_test_vap-R2_test_T # negative because high R2 the better
  predictions = (float(XP[idx])-T_c)**2/T_c**2 + (float(P_pred[idx])-P_c)**2/P_c**2
  predictions += ((float(rho_liq_pred[idx]) - rho_c)**2 + (float(rho_vap_pred[idx]) - rho_c)**2)/rho_c**2

  return min_corr + predictions

In [ ]:
N_x = 3
bounds = [(0.1, 10)]*N_x;
hidden_layers1 = [60, 80, 100, 120, 140]
hidden_layers2 = [1, 10, 20, 40]
integers_dict = {
    'liq_hidden1': hidden_layers1,
    'liq_hidden2': hidden_layers2,
    'P_hidden1': hidden_layers1,
    'P_hidden2': hidden_layers2,
    'vap_hidden': hidden_layers1,
    }

budget = 100

t0 = time.time()
dummy = optimizer_discrete(physical_ML, N_x, bounds, integers_dict, budget)
t = time.time()
assert (t - t0 < 300), "Ensure your optimizer finishes in 5 minutes"

print(f"Your optimizer reached an optimum of {dummy:.3f}")

R squared value on liq test set: -16.657
R squared value on vap test set: 0.918
R squared value on T test set: 0.793
Predicted critical point: T - 272.0 ; P - 4728667 ; rho_liq - 2174 ; rho_vap - 2047
Real critical point: T 304.17 - P - 7383113 ; rho_liq - 9596 ; rho_vap - 9596

Your optimizer reached an optimum of 16.303


### Test functions (10 marks)

##B) Reduced-order optimization (40 marks)

As you might have already noticed, DFO excels in low dimensions ($n_d$), where there are only few degrees of freedom in the decision variables that map the objective through an expensive black-box. As a rule of thumb, DFO solvers are useful up to around 100 dimensions. This is especially problematic for model-based derivative-free optimization solvers where typically at least $\mathcal{O}(n_d)$ are needed to construct meaningful surrogates for simple linear surrogates.

However, many problems present particular mathematical structures that can be exploited to address the curse of dimensionality. These problems present a true / intrinsic / effective dimensionality that is less than the 'natural' dimensionality $n_e \leq n_d$. These problems arise in hyperparameter estimation for example, where changes in only a few dimensions explains much of the variability in the objective. These functions are also said to have 'low effective dimensionality', or 'active subspaces'. An example in chemical engineering might be the following: Imagine you want to use derivative-free optimization to (among other variables) find the optimal set of flow of pure component A ($F_A$) and B ($F_B$) into a reactor. In this case, although you theoretically have complete freedom over both flows, in practice, you could optimize the ratio of A to B instead.

The following [paper](https://academic.oup.com/imaiai/article/11/1/167/6278168) has a nice introduction and motivating case. There is no need to read through the theoretical derivations.

An alternative to the iterative random embeddings presented in the previous approach would be to use partial least squares or other dimensionality reduction technique to find the lower-order embeddings.

Another approach would be to use statistical methods (either after an initial space-filling design, or iteratively) to decide which dimensions are most important and worth 'fine-tuning'. This framework of considering a subset (or linear combinations) of  the orginal dimensions could also be employed to direct DFO methods (for example coordinate search). [[Other example]](https://link.springer.com/article/10.1007/s11590-016-1028-2)






rosenbrock_() - x can have any arbitrary dimension $n_d$, but only the first 2 matter ($n_e = 2$)

rosenbrock_higher() - takes x with $n_d$ dimensions and constructs $n_d$ new dimensions as a linear combination of the original $n_d$ while preserving the effective dimensionality of $n_e = 2$

In [ ]:
from scipy.stats import ortho_group

In [ ]:
#@title High-dimensional Rosenbrock with low effective dimensionality (16 marks)
def rosenbrock_(x, seed=0):
    x = np.array(x).squeeze()
    assert x.ndim == 1
    return 100*(x[1]-x[0]**2)**2 + (x[0]-1)**2

def rosenbrock_higher(Q, x):
    # Q is used to 'mix up' the fake and effective dimensions
    x = np.array(x).squeeze()
    assert x.ndim == 1
    dim = len(x)
    # np.random.seed(seed)
    # Q = ortho_group.rvs(len(x))
    x = x.reshape(-1,1)

    new_x = (Q @ x).squeeze()
    return rosenbrock_(new_x)

Construct an optimizer that in addition to the optimizer() inputs also receives $n_e$ and leverages this knowledge to perform sample-efficient reduced-order optimization

In [ ]:
def optimizer_reduced(f, N_x: int, bounds: List[Tuple[float]], n_e, N: int = 100) -> float:
  '''
  Optimizer aims to optimize a black-box function 'f' using the dimensionality
  'N_x', and box-'bounds' on the decision vector
  Input:
    f: function: taking as input a list of size N_x and outputing a float
    N_x: int: number of dimensions
    n_e: int: number of effective/active/true dimensions, usually << n_e
    N: int: optional: Evaluation budget
    bounds: List of size N where each element i is a tuple conisting of 2 floats
            (lower, upper) serving as box-bounds on the ith element of x
  Return:
    lowest value found for f, f_min
  '''
  if N_x != len(bounds):
    raise ValueError('Nbr of variables N_x does not match length of bounds')

  ### Your code here
  x = [np.mean(bounds[i]) for i in range(N_x)]
  f_best = f(x)
  ###

  return f_best

Training benchmarks on increasing levels of Rosenbrock

In [ ]:
dims = [3, 5, 10, 20, 50, 100, 200, 400]
budgets = [50, 100, 100, 200, 500, 1000, 2000, 4000]

all_results = {'benchmark1': [], 'benchmark2': [], 'benchmark3': [], 'dummy': [],}

N_e = 2

for budget, N_dim in zip(budgets, dims):

  print(f"{budget} evaluations for {N_dim} dimensions")

  np.random.seed(0)
  Q = ortho_group.rvs(N_dim)
  wrapper_RB = partial(rosenbrock_higher, Q)

  x0 = np.array([0]*N_dim)
  bounds = [(-.5, .5)]*N_dim

  benchmark1 = minimize(
    wrapper_RB, x0,
    bounds=bounds,
    method='Nelder-Mead',
    options={'maxfev': budget},
    )

  benchmark2 = minimize(
    wrapper_RB, x0,
    bounds=bounds,
    method='Powell',
    options={'maxfev': budget},
    )

  benchmark3 = minimize(
    wrapper_RB, x0,
    bounds=bounds,
    method='COBYLA',
    options={'maxiter': budget},
    )

  t0 = time.time()
  dummy = optimizer_reduced(wrapper_RB, N_dim, bounds, N_e, budget)
  t = time.time()
  assert (t - t0 < 300), "Ensure your optimizer finishes in 5 minutes"

  all_results['benchmark1'] += [benchmark1.fun]
  all_results['benchmark2'] += [benchmark2.fun]
  all_results['benchmark3'] += [benchmark3.fun]
  all_results['dummy'] += [dummy]

print(f"Optima reached {all_results}")


50 evaluations for 3 dimensions
100 evaluations for 5 dimensions
100 evaluations for 10 dimensions
200 evaluations for 20 dimensions
500 evaluations for 50 dimensions


/usr/local/lib/python3.7/dist-packages/scipy/optimize/_minimize.py:545: RuntimeWarning: Method COBYLA cannot handle bounds.
  RuntimeWarning)


1000 evaluations for 100 dimensions
2000 evaluations for 200 dimensions
4000 evaluations for 400 dimensions
Optima reached {'benchmark1': [0.7893822376755726, 0.6532304531584366, 0.9090875086165523, 0.9780590509232587, 0.9951491302804742, 0.9976153406653555, 0.9985613957359483, 0.9990726294010398], 'benchmark2': [0.4463858580365516, 0.29352168617463, 0.26251068649162607, 0.16242376816983514, 0.07773778283758705, 0.005552736739907255, 2.9466241542255258e-05, 1.1902632184615152e-13], 'benchmark3': [0.286536988093545, 0.250675852293827, 0.3643126419916998, 0.07004675739374235, 0.04490218582897468, 8.968359627701061e-09, 4.2940679819355594e-13, 2.1302723184459655e-14], 'dummy': [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]}


Let's optimize a 376-dimensional derivative-free optimization problem to solve a supply chain planning problem

In [ ]:
#@title Simulation data

products = ["PA", "PB", "PC", "PD", "TEE", "TGE"]
secondary_sites = ["Asia", "America"]
resources = ["Granulates", "Compress", "Coat", "QC", "Packing"]
# intermediates = ['TA', 'TB', 'I1', 'I2', 'I3', 'I4']
T_set = np.arange(1, 25).tolist()


# F = {("PA", t): (t - 1) * 4000 / 100 + 5000 for t in T_set}
# F_ = {("PC", t): (t - 1) * 1e4 / 100 + 10000 for t in T_set}
# F__ = {("TEE", t): (t - 1) * 1e5 / 100 + 400000 for t in T_set}
F = {("PA", t): 5000*(1+.1*int(t>=5) - .2*int(t>=10) + .3*int(t>=15) - .4*int(t>=20)) for t in T_set}
F_ = {("PC", t): 10000*(1+.1*int(t>=5) - .2*int(t>=10) + .3*int(t>=15) - .4*int(t>=20)) for t in T_set}
F__ = {("TEE", t): 300000*(1+.1*int(t>=5) - .2*int(t>=10) + .3*int(t>=15) - .4*int(t>=20)) for t in T_set}
F_TGE = {("TGE", t): 300000*(1-.1*int(t>=5) + .2*int(t>=10) - .3*int(t>=15) + .4*int(t>=20)) for t in T_set}
F_PB = {("PB", t): 5000*(1-.1*int(t>=5) + .2*int(t>=10) - .3*int(t>=15) + .4*int(t>=20)) for t in T_set}
F_PD = {("PD", t): 10000*(1-.1*int(t>=5) + .2*int(t>=10) - .3*int(t>=15) + .4*int(t>=20)) for t in T_set}
F.update(F_) ; F.update(F__) ; F.update(F_TGE)
F.update(F_PB) ; F.update(F_PD)

data = { None: {
        "P": {None: products},
        "L": {None: secondary_sites},
        "R": {None: resources},
        "T": {None: T_set},
        # Initial storage
        "S0": {
            "PA": 15e3, "PB": 15e3,
            "PC": 6e4, "PD": 6e4,
            "TEE": 2e6, "TGE": 2e6},
        "SAIP0": {None: 3000},
        "SI0": {None: 3000},
        "SAIS0": {"Asia": 480, "America": 360},
        #Safety storage
        "IAIPstar0": {None: 3000},
        "IIstar0": {None: 3000},
        "Istar0": {
            "PA": 15e3, "PB": 15e3, "PC": 6e4, "PD": 6e4,
            "TEE": 2e6, "TGE": 2e6,
            },
        "IAISstar0": {"Asia": 400, "America": 300},
        #Costs
        "CT": {"Asia": 15, "America": 10}, #Transport
        "CS": {"PA": 0.1, "PC": 0.15, "TEE": 0.1,
                "PB": 0.09, "PD": 0.16, "TGE": 0.1,}, # Storage
        "CS_SAIS": {"Asia": 0.02, "America": 0.03},
        # 'CS_SAIS': {'Asia': 0, 'America': 0},
        "CS_AIP": {None: 0.02},
        "CS_I": {None: 0.01},
        # 'CS_AIP': {None: 0}, 'CS_I': {None: 0},
        "RM_Cost": {None: 0.1}, #Raw material cost
        "AI_Cost": {None: 0.5}, # AI cost (should not matter for the whole company)
        # 'RM_Cost': {None: 0}, 'AI_Cost': {None: 0},
        "Price": {"PA": 10, "PB": 10, "PC": 10, "PD": 10,
                  "TEE": 1, "TGE": 1,},
        "CP": {"PA": 1.2, "PC": 1.2, "TEE": 0.06,
               "PB": 1.19, "PD": 1.21, "TGE": 0.06,}, # Production
        "CP_I": {None: 0.05e-1},
        "CP_AI": {None: 0.2e-1},
        "SP_AI": {None: 2500}, # Selling price
        # 'SP_AI': {None: 0},
        "LT": {"Asia": 4, "America": 3}, # lead time
        "Q": {"PA": 20e-6 * 12 / 0.02, "PC": 6e-4 / 0.02, "TEE": 3e-5,
              "PB": 21e-6 * 12 / 0.02, "PD": 5.9e-4 / 0.02, "TGE": 3.1e-5,}, # AI to material conversion
        # No FPAI(T), FPTA(T), FPTB(T), FPI4(T), FPI3(T), FPI2(T), FPI1(T), FTP(T,K)
        "A": {  # Resource availability
            ("Asia", "Granulates"): 120*2,
            ("Asia", "Compress"): 480*2,
            ("Asia", "Coat"): 480*2,
            ("Asia", "QC"): 1800*2,
            ("Asia", "Packing"): 320*2,
            ("America", "Granulates"): 120*2,
            ("America", "Compress"): 120*2,
            ("America", "Coat"): 120*2,
            ("America", "QC"): 720*2,
            ("America", "Packing"): 160*2,
        },
        "U": { # Resource time utilised per product
            ("Asia", "Granulates"): 133e-3,
            ("Asia", "Compress"): 350e-3,
            ("Asia", "Coat"): 333e-3,
            ("Asia", "QC"): 2,
            ("Asia", "Packing"): 267e-3,
            ("America", "Granulates"): 133e-3,
            ("America", "Compress"): 350e-3,
            ("America", "Coat"): 333e-3,
            ("America", "QC"): 2,
            ("America", "Packing"): 267e-3,
        },
        "CCH": {
            ("PA", "PA"): 0,
            ("PA", "PB"): 3e4,
            ("PA", "PC"): 1e8,
            ("PA", "PD"): 1e8,
            ("PA", "TEE"): 2.9e4,
            ("PA", "TGE"): 2.9e4,
            ("PB", "PA"): 3e4,
            ("PB", "PB"): 0,
            ("PB", "PC"): 1e8,
            ("PB", "PD"): 1e8,
            ("PB", "TEE"):2.9e4,
            ("PB", "TGE"):2.9e4,
            ("PC", "PA"): 1e8,
            ("PC", "PB"): 1e8,
            ("PC", "PC"): 0,
            ("PC", "PD"): 1e4,
            ("PC", "TEE"):1e8,
            ("PC", "TGE"):1e8,
            ("PD", "PA"): 1e8,
            ("PD", "PB"): 1e8,
            ("PD", "PC"): 1e4,
            ("PD", "PD"): 0,
            ("PD", "TEE"): 1e8,
            ("PD", "TGE"): 1e8,
            ("TEE", "PA"): 3.1e4,
            ("TEE", "PB"): 3.1e4,
            ("TEE", "PC"): 1e8,
            ("TEE", "PD"): 1e8,
            ("TEE", "TEE"): 0,
            ("TEE", "TGE"): 3e4,
            ("TGE", "PA"): 3.1e4,
            ("TGE", "PB"): 3.1e4,
            ("TGE", "PC"): 1e8,
            ("TGE", "PD"): 1e8,
            ("TGE", "TEE"): 3e4,
            ("TGE", "TGE"): 0,
        },
        "X": { # matches between final products and secondary sites
            ("Asia", "PA"): 1,
            ("Asia", "PC"): 0,
            ("Asia", "TEE"): 1,
            ("Asia", "PB"): 1,
            ("Asia", "PD"): 0,
            ("Asia", "TGE"): 1,
            ("America", "PA"): 0,
            ("America", "PC"): 1,
            ("America", "TEE"): 0,
            ("America", "PB"): 0,
            ("America", "PD"): 1,
            ("America", "TGE"): 0,
        },
        "F": F, # Demand forecast
    }
}


In [ ]:
#@title Simulation
def state_to_control_t(t, N_t, T_set):
    dummy_array = np.arange(1, 1+N_t)
    N_total = len(T_set)
    T_min = T_set[0]
    idx = 1 + (t - T_min) // int(N_total/N_t)
    return min(idx, N_t)

def simulate(Production, TP, Forecast, Sales, data, seed=0, random=True):

    N_tc = data[None]['N_t'][None]
    T_set = data[None]["T"][None]
    to_Tc = lambda t: state_to_control_t(t, N_tc, T_set)

    Storage = {}
    Demand = {}
    N_t = len(T_set)
    P_set = data[None]["P"][None]

    Demand["PA"] = np.zeros(N_t)
    Demand["PC"] = np.zeros(N_t)
    Demand["TEE"] = np.zeros(N_t)
    Demand["PB"] = np.zeros(N_t)
    Demand["PD"] = np.zeros(N_t)
    Demand["TGE"] = np.zeros(N_t)

    for t in T_set:
        for p in P_set:
            if random:
                Demand[p][t - 1] = np.random.uniform(
                    0.8 * Forecast[p][t - 1], 1.2 * Forecast[p][t - 1]
                )
            else:
                Demand[p][t - 1] = Forecast[p][t - 1]

    Storage["PA"] = np.zeros(N_t)
    Storage["PC"] = np.zeros(N_t)
    Storage["TEE"] = np.zeros(N_t)
    Storage["PB"] = np.zeros(N_t)
    Storage["PD"] = np.zeros(N_t)
    Storage["TGE"] = np.zeros(N_t)
    Storage["AIP"] = np.zeros(N_t)
    Storage["AIS_Am"] = np.zeros(N_t)
    Storage["AIS_As"] = np.zeros(N_t)
    Storage["I"] = np.zeros(N_t)

    Dummy_Sales = {}
    Dummy_Sales["PA"] = np.zeros(N_t)
    Dummy_Sales["PB"] = np.zeros(N_t)
    Dummy_Sales["PC"] = np.zeros(N_t)
    Dummy_Sales["PD"] = np.zeros(N_t)
    Dummy_Sales["TEE"] = np.zeros(N_t)
    Dummy_Sales["TGE"] = np.zeros(N_t)

    for p in P_set:
        Storage[p][0] = max(0,data[None]["S0"][p] - min(Sales[p][0], Demand[p][0]) + Production[p][0])
        Dummy_Sales[p][0] =  min(Sales[p][0], Demand[p][0], data[None]["S0"][p]+Production[p][0])
    Storage["AIS_Am"][0] = max(0,
        data[None]["SAIS0"]["America"]
        - 1.1 * (Production["PC"][0] * data[None]["Q"]["PC"] + Production["PD"][0] * data[None]["Q"]["PD"])
    )
    Storage["AIS_As"][0] = max(0,
        data[None]["SAIS0"]["Asia"]
        - 1.1
        * (
            Production["PA"][0] * data[None]["Q"]["PA"]
                    + Production["TEE"][0] * data[None]["Q"]["TEE"] +
                    Production["PB"][0] * data[None]["Q"]["PB"]
                    + Production["TGE"][0] * data[None]["Q"]["TGE"]
        ) # + TP["Asia"][1]
    )
    Storage["AIP"][0] = max(0,
        data[None]["SAIP0"][None] + Production["AI"][0] - (TP["America"][0] + TP["Asia"][0])
    )
    Storage["I"][0] = max(0,
        data[None]["SI0"][None] + Production["I"][0] - 1.1 * Production["AI"][0]
    )

    for t in T_set:
        for p in P_set:
            if t - 1 > 0:
                Storage[p][t - 1] = max(0,
                    Storage[p][t - 2] - min(Sales[p][t-1], Demand[p][t - 1]) + Production[p][t - 1]
                )
                Dummy_Sales[p][t-1] =  min(Sales[p][t-1], Demand[p][t-1], Storage[p][t - 2] +Production[p][0])
        t_Am = data[None]["LT"]["America"]
        t_As = data[None]["LT"]["Asia"]
        if t - 1 - t_Am >= 0:
            Storage["AIS_Am"][t - 1] = max(0,
                Storage["AIS_Am"][t - 2]
                + TP["America"][to_Tc(t-t_Am)-1]
                - 1.1 * (Production["PC"][t - 1] * data[None]["Q"]["PC"] + Production["PD"][t - 1] * data[None]["Q"]["PD"])
            )
        elif t - 1 > 0:
            Storage["AIS_Am"][t - 1] = max(0,
                Storage["AIS_Am"][t - 2] # + TP["America"]
                - 1.1 * (Production["PC"][t - 1] * data[None]["Q"]["PC"] + Production["PD"][t - 1] * data[None]["Q"]["PD"])
            )
        if t - 1 - t_As >= 0:
            Storage["AIS_As"][t - 1] = max(0,
                Storage["AIS_As"][t - 2]
                + TP["Asia"][to_Tc(t-t_As)-1]
                - 1.1
                * (
                    Production["PA"][t - 1] * data[None]["Q"]["PA"]
                    + Production["TEE"][t - 1] * data[None]["Q"]["TEE"] +
                    Production["PB"][t - 1] * data[None]["Q"]["PB"]
                    + Production["TGE"][t - 1] * data[None]["Q"]["TGE"]
                )
            )
        elif t - 1 > 0:
            Storage["AIS_As"][t - 1] = max(0,
                Storage["AIS_As"][t - 2] # + TP["Asia"]
                - 1.1
                * (
                    Production["PA"][t - 1] * data[None]["Q"]["PA"]
                    + Production["TEE"][t - 1] * data[None]["Q"]["TEE"] +
                    Production["PB"][t - 1] * data[None]["Q"]["PB"]
                    + Production["TGE"][t - 1] * data[None]["Q"]["TGE"]
                )
            )
        if t - 1 > 0:
            Storage["AIP"][t - 1] = max(0,
                Storage["AIP"][t - 2]
                + Production["AI"][t - 1]
                - (TP["America"][to_Tc(t)-1] + TP["Asia"][to_Tc(t)-1])
            )
            Storage["I"][t - 1] = max(0,
                Storage["I"][t - 2]
                + Production["I"][t - 1]
                - 1.1 * Production["AI"][t - 1]
            )
    return Storage, Demand, Dummy_Sales

In [ ]:
#@title DFO wrapper for simulation (14 marks)
def get_Forecast(data):

    T_set = data[None]["T"][None]
    N_t = len(T_set)
    F_PA = np.zeros(N_t)
    F_PB = np.zeros(N_t)
    F_PC = np.zeros(N_t)
    F_PD = np.zeros(N_t)
    F_TEE = np.zeros(N_t)
    F_TGE = np.zeros(N_t)
    for t in T_set:
        F_PA[t-1] = data[None]['F']['PA',t]
        F_PB[t-1] = data[None]['F']['PB',t]
        F_PC[t-1] = data[None]['F']['PC',t]
        F_PD[t-1] = data[None]['F']['PD',t]
        F_TEE[t-1] = data[None]['F']['TEE',t]
        F_TGE[t-1] = data[None]['F']['TGE',t]

    Forecast = {}
    Forecast['PA'] = F_PA
    Forecast['PC'] = F_PC
    Forecast['TEE'] = F_TEE
    Forecast['PB'] = F_PB
    Forecast['PD'] = F_PD
    Forecast['TGE'] = F_TGE

    return Forecast


def get_cost(input_data, Production, TP, Sales, penalization=1):

    T_set = input_data[None]["T"][None]
    N_t = len(T_set)
    N_Tc = input_data[None]["N_t"][None]

    to_Tc = lambda t: state_to_control_t(t, N_Tc, T_set)

    Forecast = get_Forecast(input_data)

    Storage, Demand, Dummy_Sales = simulate(Production, TP, Forecast, Sales, input_data, random=False)

    P_set = input_data[None]['P'][None]
    R_set = input_data[None]['R'][None]
    T_set = input_data[None]['T'][None]
    L_set = input_data[None]['L'][None]

    Price = input_data[None]['Price']
    CP = input_data[None]['CP']
    CS = input_data[None]['CS']
    CS_I = input_data[None]['CS_I'][None]
    CS_SAIS = input_data[None]['CS_SAIS']
    CS_AIP = input_data[None]['CS_AIP'][None]
    CT = input_data[None]['CT']
    CP_I = input_data[None]['CP_I'][None]
    CP_AI = input_data[None]['CP_AI'][None]

    U = input_data[None]['U']
    X = input_data[None]['X']
    A = input_data[None]['A']
    Q = input_data[None]['Q']
    IIstar0 = input_data[None]['IIstar0'][None]
    Istar0 = input_data[None]['Istar0']
    IAISstar0 = input_data[None]['IAISstar0']
    IAIPstar0 = input_data[None]['IAIPstar0'][None]

    prod_cost = sum(
        CP[p]*Production[p][t-1] for p in P_set for t in T_set
    ) + sum(
        CP_I*Production['I'][t-1] for t in T_set
    ) + sum(
        CP_AI*Production['AI'][t-1] for t in T_set
    )
    transp_cost = sum(CT[l]*TP[l][to_Tc(t)-1] for l in L_set for t in T_set)
    store_cost = np.sum(
        [CS[p] * Storage[p][t-1] for p in P_set for t in T_set]
    ) + np.sum(
        [CS_SAIS['Asia'] * Storage['AIS_As'][t-1] for t in T_set]
    ) + np.sum(
        [CS_SAIS['America'] * Storage['AIS_Am'][t-1] for t in T_set]
    ) + np.sum(
        [CS_AIP * Storage['AIP'][t-1] for t in T_set]
    ) + np.sum(
        [CS_I * Storage['I'][t-1] for t in T_set]
    )

    for p in P_set:
        for t in T_set:
            assert Storage[p][t-1] >= -1e-5

    sales = np.sum([Price[p] * Dummy_Sales[p][t-1] for p in P_set for t in T_set])

    res_viol = sum(
        max(0, U[l, r]* sum(
                Production[p][t-1]*X[l, p]*Q[p] for p in P_set
            )/A[l, r] - 1
        )**2 for l in L_set for r in R_set for t in T_set
    )

    prod_UL = sum(
        max(
            0, Production[p][t-1]/500e3-1
        )**2 for p in P_set for t in T_set
    ) + sum(
        max(
            0, Production['AI'][t-1]/500e3-1
        )**2 for t in T_set
    ) + sum(
        max(
            0, Production['I'][t-1]/500e3-1
        )**2 for t in T_set
    )

    istar_constr = 0
    for t in T_set:
        if t > 4:
            istar_constr += sum(max(- Storage[p][t-1]/Istar0[p]+1, 0)**2 for p in P_set)
            istar_constr += max(- Storage['I'][t-1]/IIstar0+1, 0)**2
            istar_constr += max(- Storage['AIS_Am'][t-1]/IAISstar0["America"]+1, 0)**2
            istar_constr += max(- Storage['AIS_As'][t-1]/IAISstar0["Asia"]+1, 0)**2
            istar_constr += max(- Storage['AIP'][t-1]/IAIPstar0+1, 0)**2
        else:
            istar_constr += sum(max(- Storage[p][t-1]/Istar0[p]+1/4, 0)**2 for p in P_set)
            istar_constr += max(- Storage['I'][t-1]/IIstar0+1/4, 0)**2
            istar_constr += max(- Storage['AIS_Am'][t-1]/IAISstar0["America"]+1/4, 0)**2
            istar_constr += max(- Storage['AIS_As'][t-1]/IAISstar0["Asia"]+1/4, 0)**2
            istar_constr += max(- Storage['AIP'][t-1]/IAIPstar0+1/4, 0)**2

    ### Add safeS, safeSI, safeSAIS, safeSAIP

    penalty = istar_constr + prod_UL + res_viol

    return (prod_cost + transp_cost + store_cost - sales)/1e6 + penalty*penalization

def wrapper(x, input_data, penalty=1):
    T_set = input_data[None]["T"][None]
    N_t = len(T_set)
    N_Tc = input_data[None]["N_t"][None]

    to_Tc = lambda t: state_to_control_t(t, N_Tc, T_set)

    P_PA = x[:N_t]
    P_PB = x[N_t:2*N_t]
    P_TEE = x[2*N_t:3*N_t]
    P_TGE = x[3*N_t:4*N_t]
    P_PC = x[4*N_t:5*N_t]
    P_PD = x[5*N_t:6*N_t]

    SA_PA =  x[6*N_t:7*N_t]
    SA_PB =  x[7*N_t:8*N_t]
    SA_TEE = x[8*N_t:9*N_t]
    SA_TGE = x[9*N_t:10*N_t]
    SA_PC =  x[10*N_t:11*N_t]
    SA_PD =  x[11*N_t:12*N_t]

    PI = x[12*N_t:13*N_t]
    PAI = x[13*N_t:14*N_t]

    TP_As = x[14*N_t:14*N_t+1*N_Tc]
    TP_Am = x[14*N_t+1*N_Tc:14*N_t+2*N_Tc]

    Production = {} ; TP = {} ; Sales = {}
    Production['PA'] = P_PA
    Production['PC'] = P_PC
    Production['TEE'] = P_TEE
    Production['PB'] = P_PB
    Production['PD'] = P_PD
    Production['TGE'] = P_TGE
    Production['AI'] = PAI
    Production['I'] = PI
    Sales['PA'] =  SA_PA
    Sales['PC'] =  SA_PC
    Sales['TEE'] = SA_TEE
    Sales['PB'] =  SA_PB
    Sales['PD'] =  SA_PD
    Sales['TGE'] = SA_TGE
    TP['Asia'] = TP_As
    TP['America'] = TP_Am

    return get_cost(input_data, Production, TP, Sales, penalization=penalty)

The best possible achievable optimum of this problem is: $f_{opt} \approx -9.43$

In [ ]:
rho = 100
Nt = 5

data_copy = data.copy()
data_copy[None].update({'N_t': {None: Nt}, 'Tc': {None: np.arange(1, 1+Nt)}})

dfo_f = lambda x: wrapper(x, data_copy, rho)

b = []
b += [(0, 20000)]*48 + [(0, 500000)]*48 + [(0, 20000)]*24 + [(0, 30000)]*24 + [(0, 10000)]*48 + [(0, 4000000)]*48
b += [(0, 15000)]*48 + [(0, 10000)]*24 + [(0, 5000)]*24 + [(0, 1000)]*5 + [(0, 2000)]*5
x0 = np.array([np.mean(list(b_)) for b_ in b])
# x0 = np.array([max(min(x_test[i]*(1+0.05*np.random.normal()),b[i][1]),0) for i in range(len(x_test))])
dim = len(b)
init_guess = dfo_f(x0)
# optimum = dfo_f(x_test)
print(init_guess)

698.4034951593749


In [ ]:
N_e = 100
budget = 10000

t0 = time.time()
test = minimize(
    dfo_f, x0,
    bounds=b,
    method='Powell',
    options={'maxfev': budget},
    )
t1 = time.time()
print(f"Test DFO optimum: {test['fun']:.3f} in {t1-t0:.3f} seconds starting from initial value of: {init_guess:.3f}")

t0 = time.time()
dummy = optimizer_reduced(dfo_f, dim, b, N_e, budget)
t = time.time()
assert (t - t0 < 300), "Ensure your optimizer finishes in 5 minutes"
print(f"Your optimum: {dummy:.3f} in {t1-t0:.3f} seconds")




Test DFO optimum: 14.963 in 69.952 seconds starting from initial value of: 698.403
Your optimum: 698.403 in -0.002 seconds


### Test functions (10 marks)

Similar test function with different data and different dimensionality